In [19]:
import os
import numpy as np
import xarray as xr
import pandas as pd
from functools import partial
from joblib import Parallel, delayed
import multiprocessing
from pathos.multiprocessing import ProcessingPool as Pool
import matplotlib.pyplot as plt
from IPython.display import display
# Load the ncdf4 package
import netCDF4 as nc
import rasterio
from shapely.geometry import Point
import geopandas as gpd
from rasterio.transform import from_origin
from rasterio.features import rasterize
from rasterio.transform import from_bounds
import rasterio
from shapely.geometry import Point
import geopandas as gpd
from rasterio.transform import from_origin
from rasterio.features import rasterize
from rasterio.transform import from_bounds
import matplotlib.colors as colors
import datetime

## File path and settings

In [20]:
# Set the file paths
ncname_cdi = "/Users/sabinmaharjan/projects/python/do/static/file/cdi_1.nc"
ncname_rain = "/Users/sabinmaharjan/projects/python/do/static/file/2024.forecast.nc"
ncname_rain_final="/Users/sabinmaharjan/projects/python/do/static/file/2.nc"
thong_3="/Users/sabinmaharjan/projects/python/do/static/file/thong/droughtoutlook_3m.nc"
thong_1="/Users/sabinmaharjan/projects/python/do/static/file/thong/droughtoutlook_1m.nc"

In [28]:
ds2 = xr.open_dataset(ncname_rain)  # Get longitude and latitude
ds2 = xr.open_dataset(ncname_rain)

# Get longitude and latitude
lon_r = ds2["lon"].values
lat_r = ds2["lat"].values

# Get rain
dname2 = "percentage_of_ensembles"
rain_array = ds2[dname2].values
nbins = 2
time = 1

# Get slices
rain_slice = rain_array[nbins - 1, time - 1, :, :]
rain_vec = np.ravel(rain_slice)

# Create DataFrame for rain data
lon_grid, lat_grid = np.meshgrid(lon_r, lat_r)
rain_df = pd.DataFrame({'lat': lat_grid.flatten(), 'lon': lon_grid.flatten(), 'rain': rain_vec})

# Clean Data (Optional)
# rain_df = rain_df.replace(np.nan, 'NA', regex=True)

# Convert time value to a datetime object
time_r = ds2["time"][time - 1].values
time_r = pd.to_datetime(time_r)

# Extract month name from datetime object
month_name = time_r.strftime('%B')
# Create a new Dataset
ds_new = xr.Dataset({
    'rain': (('lat', 'lon'), rain_slice),
    'lat': (('lat'), lat_r),
    'lon': (('lon'), lon_r),
})

# Add global attributes if needed
ds_new.attrs['title'] = 'Rainfall Data'
ds_new.attrs['description'] = 'Rainfall data extracted from ...'
# Add more attributes as needed
ds_new['time'] = (('time'), [time_r])
# Save the Dataset to a NetCDF file
output_nc_file = "/Users/sabinmaharjan/projects/python/do/static/file/2024_"+month_name+'_forecast.nc'


ds_new.to_netcdf(output_nc_file)


# 3 months

In [27]:
# Open the rain forecast data
ds2 = xr.open_dataset(ncname_rain)
rain_var = ds2["percentage_of_ensembles"]
 
# Extract the data for the first three months
first_three_months = rain_var[1, :3, :, :]
 
# Calculate the average across the first three months
rain_avg = first_three_months.mean(dim="time")
 
# Get the latitude and longitude coordinates
lats = ds2.lat
lons = ds2.lon
 
# Create a new Dataset with the averaged rain data
ds_avg = xr.Dataset(
    {
        "rain": (("lat", "lon"), rain_avg.values),
    },
    coords={"lat": lats, "lon": lons},
)
ds_avg['time'] = (('time'), [time_r])
# Save the new Dataset as a NetCDF file
out_ncname = "/Users/sabinmaharjan/projects/python/do/static/file/2024_"+month_name+'_avg_forecast.nc'
ds_avg.to_netcdf(out_ncname)

print(f"Averaged rain forecast file saved as: {out_ncname}")

Averaged rain forecast file saved as: /Users/sabinmaharjan/projects/python/do/static/file/2024_May_avg_forecast.nc
